<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [2]:
import numpy as np

In [3]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [4]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [22]:
docs = []
vocab = set()
for i in corpus:
    terminos = i.split(' ')
    docs.append(terminos)
    vocab.update(set(terminos))

print(f'Corpus: {docs}')
print(f'Vocabulario: {vocab}')

Corpus: [['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]
Vocabulario: {'muchas', 'gracias', 'que', 'es', 'de', 'hoy', 'martes', 'el', 'dia'}


### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [91]:
my_corpus = docs
my_vocab = list(vocab)     # convierto a lista para asegurar orden de los terminos
n_docs = len(my_corpus)
n_vocab = len(my_vocab)
print(my_corpus)
print(my_vocab)

ohe = np.zeros(shape=(n_docs, n_vocab))
for j, term in enumerate(my_vocab):
    # para cada termino en mi vocabulario, busco si esta presente en cada documento
    for i in range(n_docs):
        ohe[i, j] = 1 if term in my_corpus[i] else 0

print('Matriz de vectores One Hot Encoding:')
print(ohe)

[['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]
['muchas', 'gracias', 'que', 'es', 'de', 'hoy', 'martes', 'el', 'dia']
Matriz de vectores One Hot Encoding:
[[0. 0. 1. 1. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 0. 0.]]


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [54]:
my_corpus = docs
my_vocab = list(vocab)     # convierto a lista para asegurar orden de los terminos

n_docs = len(my_corpus)
n_vocab = len(my_vocab)
print(my_corpus)
print(my_vocab)

freq_vector = np.zeros(shape=(n_docs, n_vocab))
for j, term in enumerate(my_vocab):
    for i in range(n_docs):
        freq_vector[i, j] = my_corpus[i].count(term)

print('Matriz de vectores de frecuencia:')
print(freq_vector)

[['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]
['muchas', 'gracias', 'que', 'es', 'de', 'hoy', 'martes', 'el', 'dia']
[[0. 0. 1. 1. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 1. 1. 2. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 0. 0.]]


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [89]:
my_corpus = docs
my_vocab = list(vocab)     # convierto a lista para asegurar orden de los terminos

n_docs = len(my_corpus)
n_vocab = len(my_vocab)

# Creo un diccionario con el IDF de cada termino
idf_dict = dict()
for t in my_vocab:
    df = 0
    for doc in my_corpus:
        if t in doc:
            df += 1
    idf_dict[t] = np.log10(n_docs / df)

# Ahora creo la tabla TFIDF
tfidf_matrix = np.zeros(shape=(n_docs, n_vocab))
for j, term in enumerate(my_vocab):
    for i in range(n_docs):
        tfidf_matrix[i, j] = my_corpus[i].count(term) * idf_dict[term]
print('Matriz TF-IDF:')
print(tfidf_matrix)


Matriz TF-IDF:
[[0.         0.         0.47712125 0.17609126 0.         0.17609126
  0.         0.         0.17609126]
 [0.         0.         0.         0.17609126 0.47712125 0.17609126
  0.35218252 0.47712125 0.17609126]
 [0.47712125 0.47712125 0.         0.         0.         0.
  0.17609126 0.         0.        ]]


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [94]:
# Esta funcion acepta como argumento una matriz de texto vectorizado y el indice de un documento de referencia. 
# A partir del documento de referenciam, obtiene el vector (ya sea vector de frecuencia, one-hot encoding o TF-IDF), 
# y computa la similitud coseno entre ese vector y el resto de los documentos. 
# Devuelve un arreglo de indices de documentos, ordenados de mayor a menor similitud. El primer valor siempre será el
# valor ref_doc_index, ya que la similitud coseno de un documento consigo mismo siempre dará 1.
# Nota: No se computa la matriz de texto vectorizado en esta función para mantenerla lo más simple posible. Cómo obtener
# esta matriz ya se implementó en los ejercicios anteriores.
def ordenar_por_similitud_coseno(df, ref_doc_index):
    n_docs = df.shape[0]
    vector = np.zeros(shape=(1,n_docs))
    v1 = df[ref_doc_index, :]     # obtengo vector para del doc de referencia
    for other_doc_idx in range(n_docs):
        v2 = df[other_doc_idx, :]    # obtengo vector para doc a comparar
        vector[0,other_doc_idx] = cosine_similarity(v1, v2)
    print(f'Vector de similitud coseno: {vector}')

    return np.argsort(-vector).flatten()  # devuelvo los indices en orden descendiente de similitud coseno

# Vemos como se compara el resultado con cada tipo de matriz de vectores
print(ordenar_por_similitud_coseno(tfidf_matrix, 1))
print(ordenar_por_similitud_coseno(ohe, 1))
print(ordenar_por_similitud_coseno(freq_vector, 1))

Vector de similitud coseno: [[0.2003419  1.         0.10845712]]
[1 0 2]
Vector de similitud coseno: [[0.61237244 1.         0.23570226]]
[1 0 2]
Vector de similitud coseno: [[0.5        1.         0.38490018]]
[1 0 2]
